In [1]:
total_space_in_cm = int(input("Enter the total shelfspace in cm. of the area: "))
print(f"You entered: {total_space_in_cm}")

Enter the total shelfspace in cm. of the area: 77061
You entered: 77061


In [2]:
total_occupied_space_in_cm = int(input("Enter the total occupied shelfspace in cm. of the area: "))
print(f"You entered: {total_occupied_space_in_cm}")

Enter the total occupied shelfspace in cm. of the area: 67633
You entered: 67633


In [3]:
total_free_space_in_cm = total_space_in_cm - total_occupied_space_in_cm
print(f"Total free space: {total_free_space_in_cm}")

Total free space: 9428


In [4]:
!pip install ace_tools


In [14]:
import pandas as pd

# Upload the necessary data files into Colab in the file content\files\inputs
#Maps of all shelves and banks of shelving are saved at \\san\library\Collections Management\Space\SHELVING MAPS AND MEASUREMENTS\Geier Library - July 2024\Geier
#Accuracy of the allocations depends on the accuracy of the lengths and numbers of shelves and the measurements of occupied space.
#Choose the appropriate tab from the shelf and bank maps, save separately, edit so that it contains only shelf ID and length (cm.) columns, and save locally as "shelf_map"
#Run in Alma analytics, tweaking filters and formulae where necessary,the report BER 2024 item counts by subclass and FY - GEIER
#Edit the report so that it contains only columns subclass and growth as proportion of all FY20-24 volumes, then manually add a column "total occupied" with cm occupied by each subclass, from ad-hoc measurements or shelf map subtotals. Save locally as "subclass_data"

#In the content folder, for every new runtime session you have to create a folder called files and a folder inside it called inputs to create the filepath content\files\inputs in Colab
#Read in the Excel files
shelf_map_df = pd.read_excel('/content/files/inputs/shelf_map.xlsx')
subclass_data_df = pd.read_excel('/content/files/inputs/subclass_data.xlsx')

# Replace blank values in subclass_data with 0
subclass_data_df = subclass_data_df.fillna(0)

# Calculate the free space for each subclass
subclass_data_df['free space per subclass'] = subclass_data_df['growth as proportion of all FY20-24 volumes'] * total_free_space_in_cm

# Calculate the total allocated space, occupied and free, for each subclass
subclass_data_df['total occupied and free'] = subclass_data_df['current occupied shelfspace in cm.'] + subclass_data_df['free space per subclass']

#Calculate the proportion of free space to total space per subclass
subclass_data_df['proportion free space of total space per subclass'] = subclass_data_df['free space per subclass'] / subclass_data_df['total occupied and free']

# Initialize variables for shelf assignment
shelf_allocation = []
current_shelf_start = 0
current_cumulative_space = 0

# Calculate cumulative space for each subclass
subclass_data_df['cumulative_space'] = subclass_data_df['total occupied and free'].cumsum()

# Assign shelves to each subclass
for index, row in subclass_data_df.iterrows():
    cumulative_space = row['cumulative_space']
    subclass = row['subclass']

    while current_shelf_start < len(shelf_map_df) and current_cumulative_space < cumulative_space:
        shelf_length = shelf_map_df.iloc[current_shelf_start]['shelf length (cm.)']
        remaining_space = cumulative_space - current_cumulative_space

        if remaining_space > 0:
            if shelf_length <= remaining_space:
                allocation_length = shelf_length
            else:
                allocation_length = remaining_space

            shelf_allocation.append({
                'subclass': subclass,
                'shelf ID': shelf_map_df.iloc[current_shelf_start]['shelf ID'],
                'shelf length (cm)': allocation_length,
                'current_cumulative_space': current_cumulative_space # Add the column here
            })
            current_cumulative_space += allocation_length
            shelf_map_df.at[current_shelf_start, 'shelf length (cm.)'] -= allocation_length

            if shelf_map_df.iloc[current_shelf_start]['shelf length (cm.)'] <= 0:
                current_shelf_start += 1

# Convert the allocation to a DataFrame
shelf_allocation_df = pd.DataFrame(shelf_allocation)

# Merge the subclass allocation data with the growth percentage data
shelf_allocation_df = shelf_allocation_df.merge(
    subclass_data_df[['subclass', 'total occupied and free', 'proportion free space of total space per subclass']],
    on='subclass',
    how='left'
)

# Calculate occupied space per shelf
shelf_allocation_df['occupied_space_per_shelf (cm)'] = shelf_allocation_df['shelf length (cm)'] * (shelf_allocation_df['total occupied and free'] / shelf_allocation_df.groupby('subclass')['shelf length (cm)'].transform('sum'))

# Calculate free space per shelf (with debugging print)
for index, row in shelf_allocation_df.iterrows():
    shelf_length = row['shelf length (cm)']
    shelf_id = row['shelf ID']
    free_space_percentage = row['proportion free space of total space per subclass']
    free_space = shelf_length * free_space_percentage
    subclass = row['subclass']
    print(f"Row {index}: shelf_id={shelf_id}, subclass={subclass}, shelf_length={shelf_length}, free_space = {free_space}")
    shelf_allocation_df.at[index, 'free_space_per_shelf (cm)'] = free_space


<ipython-input-14-301aface3860>:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '66.31135734071938' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  shelf_map_df.at[current_shelf_start, 'shelf length (cm.)'] -= allocation_length


Row 0: shelf_id=GGF-1-1, subclass=G, shelf_length=77.0, free_space = 9.036191411008094
Row 1: shelf_id=GGF-1-2, subclass=G, shelf_length=77.0, free_space = 9.036191411008094
Row 2: shelf_id=GGF-1-3, subclass=G, shelf_length=77.0, free_space = 9.036191411008094
Row 3: shelf_id=GGF-1-4, subclass=G, shelf_length=77.0, free_space = 9.036191411008094
Row 4: shelf_id=GGF-1-5, subclass=G, shelf_length=77.0, free_space = 9.036191411008094
Row 5: shelf_id=GGF-1-6, subclass=G, shelf_length=77.0, free_space = 9.036191411008094
Row 6: shelf_id=GGF-1-7, subclass=G, shelf_length=77.0, free_space = 9.036191411008094
Row 7: shelf_id=GGF-2-1, subclass=G, shelf_length=77.0, free_space = 9.036191411008094
Row 8: shelf_id=GGF-2-2, subclass=G, shelf_length=77.0, free_space = 9.036191411008094
Row 9: shelf_id=GGF-2-3, subclass=G, shelf_length=77.0, free_space = 9.036191411008094
Row 10: shelf_id=GGF-2-4, subclass=G, shelf_length=77.0, free_space = 9.036191411008094
Row 11: shelf_id=GGF-2-5, subclass=G, shel

In [15]:
import os

# Specify the output directory and file name
output_dir = '/content/files/outputs'
output_file = os.path.join(output_dir, 'output.xlsx')

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Export to Excel
shelf_allocation_df.to_excel(output_file, index=False)

print(f'DataFrame has been exported to {output_file}')

DataFrame has been exported to /content/files/outputs/output.xlsx
